In [ ]:
import numpy as np 
from scipy.special import k1
import pandas as pd 
from scipy.stats import norminvgauss 
from tqdm import tqdm 

def sample_nig(num , alpha , beta , gamma , mu ): 
    return norminvgauss.rvs(alpha*gamma , beta*gamma , mu , gamma , size = num  ) 

def pdf_nig(x , alpha, beta , gamma , mu ): 
    return norminvgauss.pdf(x  , alpha*gamma  , beta*gamma , mu , gamma  )

def pdf_nig_man(x , alpha , beta , gamma , mu ): 
    lambda_ = np.sqrt(alpha ** 2 - beta ** 2 ) 
    part1 = alpha * gamma * k1(alpha * np.sqrt( gamma ** 2 + (x- mu)**2))
    part2 = np.pi * np.sqrt(gamma**2 + (x-mu) ** 2 )
    part3 = np.exp(gamma * lambda_ + beta* (x- mu  )) 
    return (part1/ part2 ) * part3 
print(pdf_nig(0.1 , 2 , 0.2 , 0.8 , 0.04 ))
print(pdf_nig_man(0.1 , 2 , 0.2 , 0.8 , 0.04 ))
import numpy as np
from scipy.special import kv  # modified Bessel Kν

def p_prime(x, alpha, beta, gamma, mu):
    delta = gamma                   # follow the user's naming
    t = x - mu
    R = np.sqrt(delta**2 + t**2)
    z = alpha * R

    K0 = kv(0, z)
    K1 = kv(1, z)
    K2 = kv(2, z)

    big_gamma = np.sqrt(alpha**2 - beta**2)   # γ = √(α²−β²)
    C = (alpha * delta / np.pi) * np.exp(delta * big_gamma + beta * t)

    term1 = beta * K1 / R
    term2 = t / R**3 * (-(alpha * R * (K0 + K2) / 2) - K1)

    return C * (term1 + term2)


def payoff_fn(x , K ): 
    return 50 * np.maximum( np.exp(x) - K  , 0 ) 
def H1(x , K , theta , alpha , beta , gamma , mu ): 
    part1 = np.exp(-2 * np.abs(theta)) 
    part2 = payoff_fn(x , K )**2 
    part3 = p_prime(x - 2*theta , alpha , beta , gamma , mu ) / pdf_nig(x , alpha , beta , gamma , mu ) 
    part4 = (pdf_nig(x - theta , alpha , beta , gamma , mu   ) / pdf_nig(x - 2*theta , alpha , beta , gamma , mu ))**2 
    return part1 * part2 * part3 * part4 

def phi(theta , alpha , beta , gamma , mu ): 
    return mu*theta + gamma * (np.sqrt(alpha ** 2 - beta ** 2 ) - np.sqrt(alpha ** 2  - (beta + theta )**2 )) 
def phi_prime(theta, alpha, beta, gamma, mu):
    return mu + gamma * (beta + theta) / np.sqrt(alpha**2 - (beta + theta)**2)

def T(theta, alpha , beta ): 
    return ( beta - alpha )* theta / np.sqrt(1 + theta**2 )

def T_prime(theta , alpha , beta ): 
    return (beta - alpha )/ (np.sqrt(1 +theta**2 ) ** 3 )
def H2( x , K , theta , alpha , beta , gamma , mu ): 
    part1 = np.exp(-np.abs(theta )) 
    part2 = payoff_fn(x , K)**2 
    part3 = (phi_prime(theta , alpha , beta , gamma , mu ) - x )

    return part1 * part2 * part3

    
def get_theta_translation(theta_0 , K , num_iter , alpha , beta , gamma , mu ): 
    samples = sample_nig(num_iter , alpha , beta , gamma , mu ) 
    for i in tqdm(range(num_iter)): # on ajout le calcul par T pour stabilise 
        #theta = T(theta_0 , alpha , beta )
        step = 1 / (1000 + i )
        #theta_0 -= step * T_prime(theta_0 , alpha, beta ) *  H1(samples[i] , K , theta, alpha , beta , gamma, mu )  
        theta_0 -= step  *  H1(samples[i] , K , theta_0, alpha , beta , gamma, mu )  

    #return T(theta_0 , alpha , beta ) 
    return theta_0 
def get_theta_escher(theta_0 , K , num_iter , alpha , beta , gamma , mu ): 
    for i in tqdm(range(num_iter)) : 
        theta = T(theta_0 , alpha, beta )
        sample = sample_nig(1 , alpha , beta - theta , gamma , mu )[0] 
        step = 1 / (1000 + i ) 
        theta_0 -= step * T_prime(theta_0 , alpha , beta )* H2(sample , K , theta , alpha , beta , gamma , mu )

    return T(theta_0 , alpha , beta )  

def get_theta_escher(theta_0 , K , num_iter , alpha , beta , gamma , mu ): 
    for i in tqdm(range(num_iter)) : 
        theta = T(theta_0 , alpha, beta )
        sample = sample_nig(1 , alpha , beta - theta , gamma , mu )[0] 
        x_s = np.clip(sample, -700.0, 700.0)
        step = 1 / (1000 + i ) 
        theta_0 -= step * T_prime(theta_0 , alpha , beta )* H2(x_s , K , theta , alpha , beta , gamma , mu )

    return T(theta_0 , alpha , beta )  
def simulate_mc_crude(num_iter , strike  , alpha , beta , gamma , mu ) :
    value_0 = 0 
    samples = sample_nig(num_iter , alpha , beta , gamma , mu )
    for i in range(num_iter): 
        value_0 += payoff_fn(samples[i] , strike) 

    return value_0/num_iter  

simulate_mc_crude( 10000 , 1 , 2 , 0.2 , 0.8 , 0.04 )
thet_escher = get_theta_escher(0.3 , 1 , 100000 , 2 , 0.2 , 0.8 , 0.04 )
thet_escher
thet_trans = get_theta_translation(0.3, 1 , 100000 , 2 , 0.2 , 0.8 , 0.04 )
thet_trans
def MC_translation(M  , K , theta_opt , alpha , beta , gamma , mu ) :  
    val_ = 0 

    for i in tqdm(range(M)  ) : 
        sample = sample_nig(K , alpha , beta , gamma , mu )[0] 
        val_ += payoff_fn(sample , K ) *  pdf_nig(sample  + theta_opt , alpha , beta , gamma , mu ) / pdf_nig(sample  + theta_opt , alpha , beta , gamma , mu )

    return val_/M
price = MC_translation(100000 , 1 , thet_trans , 2 , 0.2  , 0.8 , 0.04 )
price
import numpy as np
from tqdm import tqdm

def MC_esscher(M, K, theta, alpha, beta, gamma, mu):
    # precompute the cumulant psi(θ) once and clip it
    psi_val = np.clip(phi(theta, alpha, beta, gamma, mu), -700.0, 700.0)
    assert abs(beta + theta) < alpha, "θ outside NIG CGF domain"

    total = 0.0
    for _ in tqdm(range(M)):
        # draw under the Esscher tilt
        x = np.squeeze(sample_nig(1, alpha, beta + theta, gamma, mu))

        # clip x so no inner exp() blows up
        x_s = np.clip(x, -700.0, 700.0)

        # stable combined payoff×weight:
        #   50*(e^x - K)*e^{-θ x}:
        # = 50*(e^{(1-θ)x} - K e^{-θ x})
        a1 = (1.0 - theta) * x_s
        a2 = -theta * x_s

        e1 = np.exp(np.clip(a1, -700.0, 700.0))
        e2 = np.exp(np.clip(a2, -700.0, 700.0))

        total += 50.0 * (e1 - K * e2)

    # average and then apply e^{ψ(θ)}
    return (total / M) * np.exp(psi_val)

price_escher = MC_esscher(100000 , 1,thet_escher   , 2 , 0.2 , 0.8 , 0.04 )

In [24]:
price_escher

21.366497418283494

In [3]:


import numpy as np
from scipy.special import k1, kv # kv needed for p_prime
from scipy.stats import norminvgauss
from tqdm import tqdm
import warnings

# Suppress potential warnings from extreme values in intermediate calculations
warnings.filterwarnings("ignore", category=RuntimeWarning)

# --- Basic NIG Functions (Keep as is) ---
def sample_nig(num, alpha, beta, gamma, mu):
    """Samples from the Normal Inverse Gaussian distribution."""
    # scipy uses delta (gamma) and a*delta, b*delta parameterization
    # Ensure valid parameters
    if alpha <= abs(beta):
      raise ValueError("NIG condition alpha > |beta| violated.")
    if gamma <= 0:
        raise ValueError("NIG condition gamma > 0 violated.")
    if alpha <= 0:
        raise ValueError("NIG condition alpha > 0 violated.")

    # Parameter transformation for scipy's norminvgauss
    # scipy uses a = alpha*gamma, b = beta*gamma, loc=mu, scale=gamma
    a_scipy = alpha * gamma
    b_scipy = beta * gamma
    loc_scipy = mu
    scale_scipy = gamma
    return norminvgauss.rvs(a_scipy, b_scipy, loc=loc_scipy, scale=scale_scipy, size=num)

def pdf_nig(x, alpha, beta, gamma, mu):
    """Calculates the Probability Density Function (PDF) of the NIG distribution."""
    # Ensure valid parameters
    if alpha <= abs(beta):
      # Return 0 density if parameters are invalid (e.g., during optimization search)
       return np.zeros_like(x)
    if gamma <= 0:
       return np.zeros_like(x)
    if alpha <= 0:
        return np.zeros_like(x)

    # Parameter transformation for scipy's norminvgauss
    a_scipy = alpha * gamma
    b_scipy = beta * gamma
    loc_scipy = mu
    scale_scipy = gamma
    # Use np.clip to avoid extreme values causing issues in pdf calculation
    x_clipped = np.clip(x, -1e10, 1e10) # Adjust bounds if necessary
    pdf_val = norminvgauss.pdf(x_clipped, a_scipy, b_scipy, loc=loc_scipy, scale=scale_scipy)
    # Replace NaN or Inf with 0, could happen with extreme x or parameters
    pdf_val = np.nan_to_num(pdf_val, nan=0.0, posinf=0.0, neginf=0.0)
    return pdf_val

# --- Derivative of PDF (Keep as is, used by H1) ---
def p_prime(x, alpha, beta, gamma, mu):
    """Calculates the derivative of the NIG PDF w.r.t. x."""
    # Ensure valid parameters
    if alpha <= abs(beta) or gamma <= 0 or alpha <= 0:
        # Return 0 if parameters are invalid
        return np.zeros_like(x)

    delta = gamma                   # follow the user's naming
    t = x - mu
    # Clip t to prevent sqrt(negative) or division by zero if delta is very small
    # And to prevent R becoming excessively large
    t_clipped = np.clip(t, -1e10, 1e10)
    R_sq = delta**2 + t_clipped**2
    # Handle potential R_sq being zero or negative if delta=0 and t=0
    if isinstance(R_sq, np.ndarray):
        R_sq[R_sq <= 0] = 1e-16 # Replace non-positive with small positive
    elif R_sq <= 0:
        R_sq = 1e-16
    R = np.sqrt(R_sq)

    z = alpha * R
    # Clip z to avoid issues in Bessel functions for very large/small args
    z_clipped = np.clip(z, 1e-10, 700) # kv overflows around 700-740

    try:
        K0 = kv(0, z_clipped)
        K1 = kv(1, z_clipped)
        K2 = kv(2, z_clipped)
    except (ValueError, FloatingPointError): # Catch potential domain/overflow errors
        return np.zeros_like(x)


    big_gamma_sq = alpha**2 - beta**2
    if big_gamma_sq <= 0: # Should not happen if alpha > |beta| holds, but safeguard
        return np.zeros_like(x)
    big_gamma = np.sqrt(big_gamma_sq)   # γ = √(α²−β²)

    # Calculate exponent term, clipping intermediate values
    exp_term_val = delta * big_gamma + beta * t_clipped
    exp_term_clipped = np.clip(exp_term_val, -700, 700) # Clip before exponentiation
    exp_part = np.exp(exp_term_clipped)

    C = (alpha * delta / np.pi) * exp_part

    # Calculate terms, avoid division by zero for R
    # Ensure R is not zero before division
    R_safe = np.where(R == 0, 1e-16, R) # Use small number if R is zero

    term1_num = beta * K1
    term1 = term1_num / R_safe

    # Check for potential numerical issues in K0+K2
    k_sum = K0 + K2
    k_sum_clipped = np.clip(k_sum, -1e100, 1e100) # Clip the sum if it becomes too large

    term2_factor1 = t_clipped / (R_safe**3)
    term2_factor2 = (-(alpha * R_safe * k_sum_clipped / 2.0) - K1)
    term2 = term2_factor1 * term2_factor2

    # Combine, handling potential NaNs or Infs
    result = C * (term1 + term2)
    result = np.nan_to_num(result, nan=0.0, posinf=0.0, neginf=0.0)
    return result

# --- Cumulant Generating Function (CGF) and its derivative (Keep as is) ---
def phi(theta, alpha, beta, gamma, mu):
    """Cumulant Generating Function (CGF) of NIG."""
    beta_plus_theta = beta + theta
    # Check domain constraint for CGF: alpha > |beta + theta|
    if alpha <= abs(beta_plus_theta):
        # Return a large value or NaN to indicate invalid theta in optimization
        # Return NaN might be better to signal failure clearly
        return np.nan
    term1 = mu * theta
    term2 = gamma * np.sqrt(alpha**2 - beta**2)
    term3 = gamma * np.sqrt(alpha**2 - beta_plus_theta**2)
    return term1 + term2 - term3

def phi_prime(theta, alpha, beta, gamma, mu):
    """Derivative of the CGF w.r.t. theta."""
    beta_plus_theta = beta + theta
    # Check domain constraint: alpha > |beta + theta|
    alpha_sq = alpha**2
    beta_plus_theta_sq = beta_plus_theta**2
    if alpha_sq <= beta_plus_theta_sq:
        # Return NaN if theta is outside the valid domain
        return np.nan
    denominator = np.sqrt(alpha_sq - beta_plus_theta_sq)
    if denominator == 0: # Avoid division by zero
       return np.sign(beta + theta) * np.inf # Or return NaN
    return mu + gamma * beta_plus_theta / denominator


# --- Parameters for the 2D problem ---
params_elec = {'alpha': 2.0, 'beta': 0.2, 'gamma': 0.8, 'mu': 0.04}
params_gas = {'alpha': 1.4, 'beta': 0.2, 'gamma': 0.2, 'mu': 0.04}
K_strike = 1.0
c_gas_factor = 1.0 # Assuming c=1, change if needed

# --- 2D Payoff Function ---
def payoff_fn_2d(x_elec, x_gas, K, c):
    """ Payoff function F(X) = 50 * max(exp(Xelec) - c*exp(Xgas) - K, 0) """
    # Clip inputs to exp to avoid overflow
    x_elec_clipped = np.clip(x_elec, -700, 700)
    x_gas_clipped = np.clip(x_gas, -700, 700)
    payoff = 50 * np.maximum(np.exp(x_elec_clipped) - c * np.exp(x_gas_clipped) - K, 0)
    return payoff

# --- 2D Sampling ---
def sample_nig_2d(num, params_e, params_g):
    """Samples independent (Xelec, Xgas) pairs."""
    x_elec = sample_nig(num, **params_e)
    x_gas = sample_nig(num, **params_g)
    return x_elec, x_gas

# --- 2D PDF ---
def pdf_nig_2d(x_elec, x_gas, params_e, params_g):
    """Calculates the joint PDF p(x_elec, x_gas) = p_e(x_elec) * p_g(x_gas)."""
    pdf_e = pdf_nig(x_elec, **params_e)
    pdf_g = pdf_nig(x_gas, **params_g)
    # Handle cases where individual pdfs might be zero to avoid 0/0 later
    # Note: If pdf_e or pdf_g is zero, the joint PDF is zero.
    return pdf_e * pdf_g

# --- 2D p_prime (Gradient of the 2D PDF w.r.t x = (x_elec, x_gas)) ---
def p_prime_2d(x_elec, x_gas, params_e, params_g):
    """ Calculates the gradient of the 2D PDF: [dp/dx_e, dp/dx_g] """
    pdf_e = pdf_nig(x_elec, **params_e)
    pdf_g = pdf_nig(x_gas, **params_g)
    p_prime_e = p_prime(x_elec, **params_e)
    p_prime_g = p_prime(x_gas, **params_g)

    grad_p_elec = p_prime_e * pdf_g
    grad_p_gas = pdf_e * p_prime_g

    return grad_p_elec, grad_p_gas


# --- H1 Function for Translation Method (2D Adaptation) ---
# H1 aims to find the optimal theta for the translation method.
# The original H1 seems related to minimizing variance E[(Payoff(X+theta))^2]
# The gradient of this w.r.t theta is E[2 * Payoff(X+theta) * Grad_Payoff(X+theta)]
# Let's adapt H1 to estimate this gradient component for the 2D case.
def H1_2d(x_elec, x_gas, K, c, theta_e, theta_g, params_e, params_g):
    """Estimates a component of the variance gradient for translation method."""
    x_e_shifted = x_elec + theta_e
    x_g_shifted = x_gas + theta_g

    payoff_val = payoff_fn_2d(x_e_shifted, x_g_shifted, K, c)

    # Need the gradient of the payoff function w.r.t. its inputs (which are the shifted x's)
    # Grad[Payoff(y_e, y_g)] = 50 * I(exp(y_e) - c*exp(y_g) - K > 0) * [exp(y_e), -c*exp(y_g)]
    # where y_e = x_e_shifted, y_g = x_g_shifted

    indicator = (np.exp(np.clip(x_e_shifted, -700, 700)) - c * np.exp(np.clip(x_g_shifted, -700, 700)) - K) > 0

    if indicator:
         # Clip intermediate exponentials
        exp_e = np.exp(np.clip(x_e_shifted, -700, 700))
        exp_g = np.exp(np.clip(x_g_shifted, -700, 700))
        grad_payoff_e = 50.0 * exp_e
        grad_payoff_g = -50.0 * c * exp_g
    else:
        grad_payoff_e = 0.0
        grad_payoff_g = 0.0

    # The stochastic gradient estimate for theta_e and theta_g
    # Proportional to payoff_val * grad_payoff_e and payoff_val * grad_payoff_g
    # Clip results to prevent explosion
    h1_e = np.clip(payoff_val * grad_payoff_e, -1e10, 1e10)
    h1_g = np.clip(payoff_val * grad_payoff_g, -1e10, 1e10)

    return h1_e, h1_g


# --- H2 Function for Esscher Transform (2D Adaptation) ---
# H2 aims to find the optimal theta for Esscher.
# Optimal theta often makes E_theta[Payoff(X) * (X - E_theta[X])] = 0
# where E_theta[X] = [phi_prime_e(theta_e), phi_prime_g(theta_g)]
def H2_2d(x_elec_tilted, x_gas_tilted, K, c, theta_e, theta_g, params_e, params_g):
    """ Calculates the stochastic gradient component for Esscher transform theta. """
    payoff_val = payoff_fn_2d(x_elec_tilted, x_gas_tilted, K, c)

    # Calculate E_theta[X] = gradient of CGF
    phi_prime_e_val = phi_prime(theta_e, **params_e)
    phi_prime_g_val = phi_prime(theta_g, **params_g)

    # Check if phi_prime returned NaN (invalid theta)
    if np.isnan(phi_prime_e_val) or np.isnan(phi_prime_g_val):
        # Return zero gradient if theta is invalid, hoping step size corrects it
        # Or could return a large penalty gradient? Zero seems safer.
        return 0.0, 0.0

    # Calculate the terms for the gradient estimate
    h2_e = payoff_val * (x_elec_tilted - phi_prime_e_val)
    h2_g = payoff_val * (x_gas_tilted - phi_prime_g_val)

    # Clip results
    h2_e = np.clip(h2_e, -1e15, 1e15) # Allow larger values than H1
    h2_g = np.clip(h2_g, -1e15, 1e15)

    return h2_e, h2_g


# --- Theta Optimization Functions (2D Adaptation) ---

def get_theta_translation_2d(theta_0_e, theta_0_g, K, c, num_iter, params_e, params_g):
    """ Finds optimal theta = (theta_e, theta_g) for translation method using SGD. """
    theta_e = theta_0_e
    theta_g = theta_0_g
    samples_e, samples_g = sample_nig_2d(num_iter, params_e, params_g)

    print("Optimizing theta for Translation Method...")
    for i in tqdm(range(num_iter)):
        step = 1.0 / (1000.0 + i) # Standard decreasing step size

        # Calculate H1 for the current sample and theta
        h1_e, h1_g = H1_2d(samples_e[i], samples_g[i], K, c, theta_e, theta_g, params_e, params_g)

        # Update theta using SGD (gradient *descent*, so subtract)
        # We are minimizing variance, related to E[Payoff^2], gradient has factor of 2, absorbed into step/scaling
        theta_e -= step * h1_e
        theta_g -= step * h1_g

        # Optional: Add clipping or constraints to theta if needed

    print(f"Translation Optimization Complete. Theta_e: {theta_e:.4f}, Theta_g: {theta_g:.4f}")
    return theta_e, theta_g

def get_theta_escher_2d(theta_0_e, theta_0_g, K, c, num_iter, params_e, params_g):
    """ Finds optimal theta = (theta_e, theta_g) for Esscher transform using SGD. """
    theta_e = theta_0_e
    theta_g = theta_0_g

    print("Optimizing theta for Esscher Transform...")
    for i in tqdm(range(num_iter)):
        step = 1.0 / (1000.0 + i) # Standard decreasing step size

        # Define parameters for the *tilted* distribution
        params_e_tilted = params_e.copy()
        params_g_tilted = params_g.copy()
        params_e_tilted['beta'] = params_e['beta'] + theta_e
        params_g_tilted['beta'] = params_g['beta'] + theta_g

        # Check if tilted parameters are valid BEFORE sampling
        valid_e = params_e_tilted['alpha'] > abs(params_e_tilted['beta'])
        valid_g = params_g_tilted['alpha'] > abs(params_g_tilted['beta'])

        if not valid_e or not valid_g:
            # If theta drifted outside domain, try to recover or stop.
            # Simple recovery: slightly nudge theta back towards zero (or original beta)
            # print(f"Warning: Theta out of bounds at iter {i}. Attempting recovery.")
            recovery_factor = 0.01 # Small nudge
            if not valid_e:
                theta_e = (1-recovery_factor) * theta_e - recovery_factor*params_e['beta']
                # Recompute tilted beta_e for this iteration
                params_e_tilted['beta'] = params_e['beta'] + theta_e
                if not (params_e_tilted['alpha'] > abs(params_e_tilted['beta'])):
                   print(f"ERROR: Recovery failed for theta_e at iter {i}. Stopping.")
                   return theta_e, theta_g # Return current (potentially bad) theta
            if not valid_g:
                theta_g = (1-recovery_factor) * theta_g - recovery_factor*params_g['beta']
                params_g_tilted['beta'] = params_g['beta'] + theta_g
                if not (params_g_tilted['alpha'] > abs(params_g_tilted['beta'])):
                    print(f"ERROR: Recovery failed for theta_g at iter {i}. Stopping.")
                    return theta_e, theta_g # Return current (potentially bad) theta
            # Proceed with potentially recovered theta for this step's calc

        # Sample from the *tilted* distribution
        try:
            sample_e_tilted, sample_g_tilted = sample_nig_2d(1, params_e_tilted, params_g_tilted)
            sample_e_tilted, sample_g_tilted = sample_e_tilted[0], sample_g_tilted[0] # Get scalar values
        except ValueError:
            # Sampling failed likely due to invalid params despite check/recovery
             print(f"ERROR: Sampling failed for tilted params at iter {i}. Stopping.")
             # Return the *last known good* thetas before the update attempt?
             # Or just the current ones.
             return theta_e, theta_g


        # Clip samples to avoid extreme values in subsequent calculations
        x_e_clipped = np.clip(sample_e_tilted, -700.0, 700.0)
        x_g_clipped = np.clip(sample_g_tilted, -700.0, 700.0)

        # Calculate H2 for the tilted sample and current theta
        h2_e, h2_g = H2_2d(x_e_clipped, x_g_clipped, K, c, theta_e, theta_g, params_e, params_g)

         # Check if H2 calculation failed (e.g., due to phi_prime being NaN)
        if np.isnan(h2_e) or np.isnan(h2_g):
            # This could happen if phi_prime failed despite earlier check
            # Skip update for this iteration
            # print(f"Warning: H2 calculation resulted in NaN at iter {i}. Skipping update.")
            continue

        # Update theta using SGD (we want E[Payoff * (X-E_theta[X])] = 0, so subtract the estimate)
        theta_e -= step * h2_e
        theta_g -= step * h2_g

    print(f"Esscher Optimization Complete. Theta_e: {theta_e:.4f}, Theta_g: {theta_g:.4f}")
    # Final check on optimized theta validity
    if not (params_e['alpha'] > abs(params_e['beta'] + theta_e) and \
            params_g['alpha'] > abs(params_g['beta'] + theta_g)):
        print("Warning: Final optimized Esscher theta is outside the valid NIG domain.")

    return theta_e, theta_g


# --- Monte Carlo Simulation Functions (2D Adaptation) ---

def simulate_mc_crude_2d(num_iter, K, c, params_e, params_g):
    """ Crude Monte Carlo simulation for the 2D payoff. """
    value_0 = 0.0
    samples_e, samples_g = sample_nig_2d(num_iter, params_e, params_g)
    print("Running Crude Monte Carlo...")
    for i in tqdm(range(num_iter)):
        value_0 += payoff_fn_2d(samples_e[i], samples_g[i], K, c)

    return value_0 / num_iter

def MC_translation_2d(M, K, c, theta_e_opt, theta_g_opt, params_e, params_g):
    """ Monte Carlo using the Translation method variance reduction. """
    # This method estimates E[Payoff(X+theta)] by sampling X from the original
    # distribution and evaluating Payoff at X+theta.
    val_ = 0.0
    samples_e, samples_g = sample_nig_2d(M, params_e, params_g)
    print("Running Monte Carlo with Translation...")
    for i in tqdm(range(M)):
        payoff_shifted = payoff_fn_2d(samples_e[i] + theta_e_opt, samples_g[i] + theta_g_opt, K, c)
        val_ += payoff_shifted

    return val_ / M

def MC_esscher_2d(M, K, c, theta_e, theta_g, params_e, params_g):
    """ Monte Carlo using the Esscher Transform importance sampling. """
    # Check Esscher parameter validity
    beta_e_tilted = params_e['beta'] + theta_e
    beta_g_tilted = params_g['beta'] + theta_g
    if not (params_e['alpha'] > abs(beta_e_tilted) and params_g['alpha'] > abs(beta_g_tilted)):
        raise ValueError("Esscher theta is outside the valid NIG domain for MC.")

    # Precompute the 2D CGF value phi(theta_e) + phi(theta_g)
    phi_e = phi(theta_e, **params_e)
    phi_g = phi(theta_g, **params_g)
    if np.isnan(phi_e) or np.isnan(phi_g):
        raise ValueError("CGF calculation failed for the given Esscher theta.")
    psi_val_2d = phi_e + phi_g
    # Clip the combined CGF value before exponentiation
    exp_psi_clipped = np.exp(np.clip(psi_val_2d, -700, 700))

    total = 0.0
    params_e_tilted = params_e.copy()
    params_g_tilted = params_g.copy()
    params_e_tilted['beta'] = beta_e_tilted
    params_g_tilted['beta'] = beta_g_tilted

    print("Running Monte Carlo with Esscher Transform...")
    # Sample from the tilted distribution
    try:
        samples_e_tilted, samples_g_tilted = sample_nig_2d(M, params_e_tilted, params_g_tilted)
    except ValueError as e:
         raise ValueError(f"Failed to sample from tilted distribution in MC_esscher_2d: {e}")


    for i in tqdm(range(M)):
        x_e = samples_e_tilted[i]
        x_g = samples_g_tilted[i]

        # Clip x samples before payoff and weight calculation
        x_e_clipped = np.clip(x_e, -700.0, 700.0)
        x_g_clipped = np.clip(x_g, -700.0, 700.0)

        # Calculate the payoff at the sampled point
        payoff_val = payoff_fn_2d(x_e_clipped, x_g_clipped, K, c)

        # Calculate the importance sampling weight: exp(-theta*x) = exp(-(theta_e*x_e + theta_g*x_g))
        weight_exponent = -(theta_e * x_e_clipped + theta_g * x_g_clipped)
        weight_exp_clipped = np.exp(np.clip(weight_exponent, -700, 700))

        # Contribution is Payoff * Weight * exp(psi(theta))
        # We multiply by exp(psi) at the end
        total += payoff_val * weight_exp_clipped
        if np.isnan(total): # Check for NaN accumulation
            print(f"Warning: NaN detected in total sum at iteration {i}. Check inputs/calculations.")
            print(f"Payoff: {payoff_val}, Weight_exp: {weight_exp_clipped}")
            # Decide how to handle: break, skip, set to 0? For now, break.
            break


    # Average and then apply the exp(psi(theta)) factor
    result = (total / M) * exp_psi_clipped
    result = np.nan_to_num(result, nan=0.0, posinf=0.0, neginf=0.0) # Final safety net
    return result


# --- Main Execution ---

# Parameters
NUM_MC_SAMPLES = 100000   # Samples for pricing
NUM_OPTIM_SAMPLES = 100000 # Samples for finding optimal theta
INITIAL_THETA_E = 0.1    # Initial guess for theta_e
INITIAL_THETA_G = 0.1    # Initial guess for theta_g

print("--- Parameters ---")
print(f"NIG Elec: {params_elec}")
print(f"NIG Gas: {params_gas}")
print(f"Strike K: {K_strike}")
print(f"Gas Factor c: {c_gas_factor}")
print(f"MC Samples for Pricing: {NUM_MC_SAMPLES}")
print(f"MC Samples for Optimization: {NUM_OPTIM_SAMPLES}")
print("-" * 20)

# 1. Crude Monte Carlo
price_crude = simulate_mc_crude_2d(NUM_MC_SAMPLES, K_strike, c_gas_factor, params_elec, params_gas)
print(f"\nCrude MC Price Estimate: {price_crude:.6f}")
print("-" * 20)

# 2. Translation Method
# 2a. Find optimal theta
theta_trans_e, theta_trans_g = get_theta_translation_2d(
    INITIAL_THETA_E, INITIAL_THETA_G, K_strike, c_gas_factor, NUM_OPTIM_SAMPLES, params_elec, params_gas
)
# 2b. Price using optimal theta
price_trans = MC_translation_2d(
    NUM_MC_SAMPLES, K_strike, c_gas_factor, theta_trans_e, theta_trans_g, params_elec, params_gas
)
print(f"\nTranslation MC Price Estimate: {price_trans:.6f}")
print(f"Optimal Translation Theta: ({theta_trans_e:.4f}, {theta_trans_g:.4f})")
print("-" * 20)

# 3. Esscher Transform
# 3a. Find optimal theta
theta_esscher_e, theta_esscher_g = get_theta_escher_2d(
    INITIAL_THETA_E, INITIAL_THETA_G, K_strike, c_gas_factor, NUM_OPTIM_SAMPLES, params_elec, params_gas
)
# 3b. Price using optimal theta
# Add a small safety check before pricing
final_beta_e = params_elec['beta'] + theta_esscher_e
final_beta_g = params_gas['beta'] + theta_esscher_g
if not (params_elec['alpha'] > abs(final_beta_e) and params_gas['alpha'] > abs(final_beta_g)):
     print("\nERROR: Optimal Esscher Theta found is outside the valid domain. Cannot price.")
     price_escher = np.nan
else:
    try:
        price_escher = MC_esscher_2d(
            NUM_MC_SAMPLES, K_strike, c_gas_factor, theta_esscher_e, theta_esscher_g, params_elec, params_gas
        )
        print(f"\nEsscher Transform MC Price Estimate: {price_escher:.6f}")
    except ValueError as e:
        print(f"\nError during Esscher MC pricing: {e}")
        price_escher = np.nan

print(f"Optimal Esscher Theta: ({theta_esscher_e:.4f}, {theta_esscher_g:.4f})")
print("-" * 20)


print("\n--- Summary ---")
print(f"Crude MC Price:        {price_crude:.6f}")
print(f"Translation MC Price:  {price_trans:.6f} (Theta: {theta_trans_e:.4f}, {theta_trans_g:.4f})")
print(f"Esscher MC Price:      {price_escher:.6f} (Theta: {theta_esscher_e:.4f}, {theta_esscher_g:.4f})")


--- Parameters ---
NIG Elec: {'alpha': 2.0, 'beta': 0.2, 'gamma': 0.8, 'mu': 0.04}
NIG Gas: {'alpha': 1.4, 'beta': 0.2, 'gamma': 0.2, 'mu': 0.04}
Strike K: 1.0
Gas Factor c: 1.0
MC Samples for Pricing: 100000
MC Samples for Optimization: 100000
--------------------
Running Crude Monte Carlo...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:01<00:00, 87613.23it/s]



Crude MC Price Estimate: 11.486584
--------------------
Optimizing theta for Translation Method...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:02<00:00, 36719.28it/s]


Translation Optimization Complete. Theta_e: -4.5065, Theta_g: 2.0400
Running Monte Carlo with Translation...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:01<00:00, 91800.13it/s]



Translation MC Price Estimate: 0.000000
Optimal Translation Theta: (-4.5065, 2.0400)
--------------------
Optimizing theta for Esscher Transform...


  5%|██████▉                                                                                                                                          | 4760/100000 [00:00<00:11, 8070.41it/s]

ERROR: Recovery failed for theta_e at iter 4760. Stopping.

ERROR: Optimal Esscher Theta found is outside the valid domain. Cannot price.
Optimal Esscher Theta: (-2.2115, 0.3957)
--------------------

--- Summary ---
Crude MC Price:        11.486584
Translation MC Price:  0.000000 (Theta: -4.5065, 2.0400)
Esscher MC Price:      nan (Theta: -2.2115, 0.3957)


In [ ]:
import numpy as np
from scipy.special import k1, kv # kv is needed for p_prime
from scipy.stats import norminvgauss
from tqdm import tqdm
import warnings

# --- Base NIG Functions (mostly unchanged) ---

def sample_nig(num, alpha, beta, gamma, mu):
    """Samples from a 1D NIG distribution."""
    # Ensure parameters are valid for norminvgauss which uses a different parametrization internally
    # scipy uses (a, b, loc, scale) corresponding to (alpha*gamma, beta*gamma, mu, gamma)
    # Check validity: alpha > 0, gamma > 0, |beta| <= alpha (scipy enforces internally)
    if alpha <= 0 or gamma <= 0:
        raise ValueError("alpha and gamma must be positive")
    # abs(beta) < alpha is required for CGF to exist for theta!=0 later
    # We allow abs(beta)==alpha here for sampling, but CGF related functions will fail
    
    # Catch potential issues with extreme parameters leading to nan/inf in rvs
    with warnings.catch_warnings():
        warnings.simplefilter("error", RuntimeWarning) # Treat warnings as errors
        try:
            samples = norminvgauss.rvs(a=alpha * gamma, b=beta * gamma, loc=mu, scale=gamma, size=num)
        except (ValueError, RuntimeWarning) as e:
            print(f"Error during sampling with params: alpha={alpha}, beta={beta}, gamma={gamma}, mu={mu}")
            print(f"Scipy params: a={alpha*gamma}, b={beta*gamma}, loc={mu}, scale={gamma}")
            raise e # Re-raise the exception
            
    # Check for NaNs/Infs which might occur even without warnings sometimes
    if np.any(np.isnan(samples)) or np.any(np.isinf(samples)):
         print(f"Warning: NaN/Inf generated during sampling with params: alpha={alpha}, beta={beta}, gamma={gamma}, mu={mu}")
         # Decide how to handle: e.g., replace with a default value or raise error
         # For now, let's replace NaNs with mu, Infs with large numbers (this might bias results)
         samples = np.nan_to_num(samples, nan=mu, posinf=mu+10*gamma*alpha, neginf=mu-10*gamma*alpha) # Simple fix, might need refinement

    return samples


def pdf_nig(x, alpha, beta, gamma, mu):
    """Calculates the PDF of a 1D NIG distribution."""
    # scipy uses (a, b, loc, scale) corresponding to (alpha*gamma, beta*gamma, mu, gamma)
    if alpha <= 0 or gamma <= 0:
         raise ValueError("alpha and gamma must be positive")
    if np.abs(beta) > alpha:
         # PDF is defined, but functions relying on CGF might fail later
         pass # Allow calculation for now
         
    # Use np.clip to avoid extreme values causing issues in pdf calculation
    x_clipped = np.clip(x, mu - 20*gamma*alpha, mu + 20*gamma*alpha) # Heuristic clipping range

    with np.errstate(divide='ignore', invalid='ignore', over='ignore'): # Suppress specific warnings
        pdf_val = norminvgauss.pdf(x_clipped, a=alpha * gamma, b=beta * gamma, loc=mu, scale=gamma)
    
    # Replace potential NaNs or Infs resulting from pdf calculation with 0
    pdf_val = np.nan_to_num(pdf_val, nan=0.0, posinf=0.0, neginf=0.0)
    pdf_val = np.maximum(pdf_val, 1e-100) # Avoid exact zeros if possible for ratios

    return pdf_val


def p_prime(x, alpha, beta, gamma, mu):
    """Derivative of the 1D NIG PDF w.r.t. x."""
    # Follows the formula provided in the original code
    delta = gamma
    t = x - mu
    R = np.sqrt(delta**2 + t**2)
    z = alpha * R

    # Avoid division by zero or issues with R=0 (when t=0 and delta=0, though delta>0 assumed)
    # Add small epsilon to R if needed, or handle t=0 case separately if delta could be 0
    R = np.maximum(R, 1e-50) # Avoid R=0
    z = np.maximum(z, 1e-50) # Avoid z=0 for Bessel functions

    # Use kv for modified Bessel function of the second kind
    # kv(nu, z) computes K_nu(z)
    K0 = kv(0, z)
    K1 = kv(1, z)
    K2 = kv(2, z)

    big_gamma_sqrt = alpha**2 - beta**2
    if big_gamma_sqrt < 0:
        # This case should not happen if |beta| < alpha, indicates invalid parameters for Esscher
        # If |beta| == alpha, big_gamma = 0. Log will handle exp(0) = 1.
         warnings.warn(f"alpha^2 - beta^2 < 0 encountered in p_prime ({alpha=}, {beta=}). May lead to NaN.")
         big_gamma = np.nan # Or handle appropriately, e.g., return 0 derivative?
    else:
         big_gamma = np.sqrt(big_gamma_sqrt)

    # Calculate C, handling potential overflows in exp
    exp_term = delta * big_gamma + beta * t
    exp_term_clipped = np.clip(exp_term, -700, 700) # Clip exponent to avoid overflow
    C = (alpha * delta / np.pi) * np.exp(exp_term_clipped)

    term1 = beta * K1 / R
    term2 = t / R**3 * (-(alpha * R * (K0 + K2) / 2) - K1)

    # Combine terms, checking for NaNs/Infs
    p_prime_val = C * (term1 + term2)
    p_prime_val = np.nan_to_num(p_prime_val, nan=0.0, posinf=0.0, neginf=0.0) # Replace NaN/Inf with 0

    return p_prime_val


def phi(theta, alpha, beta, gamma, mu):
    """Cumulant Generating Function (CGF) for 1D NIG."""
    # Check domain: alpha^2 - (beta + theta)^2 > 0 => |beta + theta| < alpha
    inner_sqrt_arg = alpha**2 - (beta + theta)**2
    if inner_sqrt_arg <= 0:
        # Theta is outside the valid domain for the CGF
        # Return NaN or Inf to indicate this
        return np.nan # Or np.inf

    term1 = mu * theta
    term2 = gamma * np.sqrt(alpha**2 - beta**2)
    term3 = gamma * np.sqrt(inner_sqrt_arg)
    
    phi_val = term1 + term2 - term3
    
    # Clip result to avoid extreme values if necessary, although CGF should be finite in domain
    phi_val = np.clip(phi_val, -np.inf, 700) # Allow negative inf, clip large positive
    
    return phi_val


def phi_prime(theta, alpha, beta, gamma, mu):
    """Derivative of the 1D NIG CGF w.r.t. theta."""
    # Check domain: alpha^2 - (beta + theta)^2 > 0 => |beta + theta| < alpha
    inner_sqrt_arg = alpha**2 - (beta + theta)**2
    if inner_sqrt_arg <= 1e-10: # Check close to zero as well for division
        # Theta is outside or on the boundary of the valid domain
        # The derivative involves 1/sqrt(arg), so it blows up at the boundary
        # Return NaN or a very large number depending on convention
        return np.nan # Or handle as +/- np.inf depending on the side?

    sqrt_term = np.sqrt(inner_sqrt_arg)
    
    phi_prime_val = mu + gamma * (beta + theta) / sqrt_term
    
    # No immediate need to clip, but be aware it can be large near boundary
    return phi_prime_val


# --- 2D Adaptations ---

def sample_nig_2d(num, params_elec, params_gas):
    """Samples independent (X_elec, X_gas) pairs."""
    x_elec = sample_nig(num, **params_elec)
    x_gas = sample_nig(num, **params_gas)
    return np.stack((x_elec, x_gas), axis=-1) # Return as shape (num, 2)

def pdf_nig_joint(x, params_elec, params_gas):
    """Calculates joint PDF for independent (X_elec, X_gas)."""
    # x is expected to be shape (..., 2)
    x_elec = x[..., 0]
    x_gas = x[..., 1]
    pdf_elec = pdf_nig(x_elec, **params_elec)
    pdf_gas = pdf_nig(x_gas, **params_gas)
    return pdf_elec * pdf_gas

def p_prime_joint(x, params_elec, params_gas):
    """Gradient of the joint PDF w.r.t. x = [x_elec, x_gas]."""
    # x is expected to be shape (..., 2)
    x_elec = x[..., 0]
    x_gas = x[..., 1]
    
    pdf_elec = pdf_nig(x_elec, **params_elec)
    pdf_gas = pdf_nig(x_gas, **params_gas)
    
    p_prime_e = p_prime(x_elec, **params_elec)
    p_prime_g = p_prime(x_gas, **params_gas)
    
    grad = np.stack((p_prime_e * pdf_gas, pdf_elec * p_prime_g), axis=-1)
    return grad # Shape (..., 2)

def phi_joint(theta, params_elec, params_gas):
    """Joint CGF for independent variables."""
    # theta is expected to be shape (..., 2)
    theta_elec = theta[..., 0]
    theta_gas = theta[..., 1]
    phi_e = phi(theta_elec, **params_elec)
    phi_g = phi(theta_gas, **params_gas)
    # If any component is NaN (outside domain), the sum is NaN
    return phi_e + phi_g

def phi_prime_joint(theta, params_elec, params_gas):
    """Gradient of the joint CGF w.r.t. theta = [theta_elec, theta_gas]."""
    # theta is expected to be shape (..., 2)
    theta_elec = theta[..., 0]
    theta_gas = theta[..., 1]
    phi_prime_e = phi_prime(theta_elec, **params_elec)
    phi_prime_g = phi_prime(theta_gas, **params_gas)
    # Result shape should match theta's shape components if input theta is multidim array
    # If theta is (2,), output is (2,)
    # If theta is (N, 2), output is (N, 2)
    grad = np.stack((phi_prime_e, phi_prime_g), axis=-1)
    return grad # Shape (..., 2)

def payoff_fn_2d(x, K, c):
    """Payoff function F(X) = 50 * max(exp(X_elec) - c*exp(X_gas) - K, 0)."""
    # x is expected to be shape (..., 2)
    x_elec = x[..., 0]
    x_gas = x[..., 1]

    # Clip inputs to exp to prevent overflow
    x_elec_clipped = np.clip(x_elec, -700, 700)
    x_gas_clipped = np.clip(x_gas, -700, 700)

    val = np.exp(x_elec_clipped) - c * np.exp(x_gas_clipped) - K
    return 50 * np.maximum(val, 0)

# --- Importance Sampling Helper Functions (Adapting H1, H2) ---
# Note: The direct adaptation of H1 is complex and its theoretical basis in 2D
# is less clear than the Esscher transform (H2).
# The H1 adaptation below attempts to mimic the structure component-wise,
# which is a simplification.

def H1_2d(x, K, c, theta, params_elec, params_gas):
    """Adapted H1 for 2D. Returns a scalar measure for simplicity."""
    # x, theta are shape (2,)
    # This adaptation is heuristic: calculate H1-like terms for each dimension
    # and combine them (e.g., take the norm or sum).
    # This avoids needing a full 2D gradient derivation of the original H1 logic.

    x_elec, x_gas = x[0], x[1]
    theta_elec, theta_gas = theta[0], theta[1]
    
    # Calculate components needed for H1 logic, per dimension
    payoff_val = payoff_fn_2d(x, K, c)
    if payoff_val == 0: # If payoff is zero, contribution to gradient should be zero
        return 0.0 # Return a scalar 0

    # --- Elec component ---
    pdf_e_x = pdf_nig(x_elec, **params_elec)
    pdf_e_x_minus_theta = pdf_nig(x_elec - theta_elec, **params_elec)
    pdf_e_x_minus_2theta = pdf_nig(x_elec - 2 * theta_elec, **params_elec)
    pprime_e_x_minus_2theta = p_prime(x_elec - 2 * theta_elec, **params_elec)

    # Avoid division by zero / handle small PDFs
    if pdf_e_x < 1e-100 or pdf_e_x_minus_2theta < 1e-100:
        h1_term_e = 0.0
    else:
        ratio_pdfs_e = pdf_e_x_minus_theta / pdf_e_x_minus_2theta
        ratio_pprime_pdf_e = pprime_e_x_minus_2theta / pdf_e_x
        h1_term_e = ( np.exp(-2 * np.abs(theta_elec)) * payoff_val**2 * 
                      ratio_pprime_pdf_e * ratio_pdfs_e**2 )

    # --- Gas component ---
    pdf_g_x = pdf_nig(x_gas, **params_gas)
    pdf_g_x_minus_theta = pdf_nig(x_gas - theta_gas, **params_gas)
    pdf_g_x_minus_2theta = pdf_nig(x_gas - 2 * theta_gas, **params_gas)
    pprime_g_x_minus_2theta = p_prime(x_gas - 2 * theta_gas, **params_gas)
    
    if pdf_g_x < 1e-100 or pdf_g_x_minus_2theta < 1e-100:
        h1_term_g = 0.0
    else:
        ratio_pdfs_g = pdf_g_x_minus_theta / pdf_g_x_minus_2theta
        ratio_pprime_pdf_g = pprime_g_x_minus_2theta / pdf_g_x
        h1_term_g = ( np.exp(-2 * np.abs(theta_gas)) * payoff_val**2 * 
                      ratio_pprime_pdf_g * ratio_pdfs_g**2 )

    # Combine: use sum of absolute values as a scalar heuristic update signal
    # A more proper approach would return a 2D vector update.
    h1_combined = np.abs(h1_term_e) + np.abs(h1_term_g) 
    
    # Clip extreme values
    h1_combined = np.nan_to_num(h1_combined, nan=0.0, posinf=1e10, neginf=-1e10)
    return np.clip(h1_combined, -1e10, 1e10) # Further clipping


def H2_2d(x, K, c, theta, params_elec, params_gas):
    """Adapted H2 for 2D (Esscher). Returns a 2D vector."""
    # x, theta are shape (2,)
    
    # Check if theta is valid for CGF calculation
    if ( np.abs(params_elec['beta'] + theta[0]) >= params_elec['alpha'] or
         np.abs(params_gas['beta'] + theta[1]) >= params_gas['alpha'] ):
        # Theta is outside domain, gradient contribution should be invalid/zero/large?
        # Returning zero vector might be safest for SGD stability if this happens often.
         warnings.warn(f"Theta {theta} outside CGF domain during H2 calculation.")
         return np.array([0.0, 0.0]) # Or np.array([np.nan, np.nan])

    payoff_val = payoff_fn_2d(x, K, c)
    if payoff_val == 0: # If payoff is zero, contribution to gradient should be zero
        return np.array([0.0, 0.0])

    phi_prime_vec = phi_prime_joint(theta, params_elec, params_gas) # Shape (2,)

    # Check if phi_prime calculation failed (e.g., theta on boundary)
    if np.any(np.isnan(phi_prime_vec)):
         warnings.warn(f"NaN encountered in phi_prime_joint for theta {theta}. H2 contribution set to zero.")
         return np.array([0.0, 0.0])

    part1_exp_term = -np.sum(np.abs(theta)) # Assuming abs element-wise then sum? Or norm? Let's use sum.
    part1 = np.exp(np.clip(part1_exp_term, -700, 700)) # Clip exponent
    part2 = payoff_val**2
    part3 = (phi_prime_vec - x) # Element-wise difference, shape (2,)

    h2_vec = part1 * part2 * part3
    
    # Clip extreme values and handle NaN/Inf
    h2_vec = np.nan_to_num(h2_vec, nan=0.0, posinf=1e10, neginf=-1e10)
    h2_vec = np.clip(h2_vec, -1e10, 1e10) # Clip vector components

    return h2_vec # Return the 2D vector


# --- Theta Optimization Functions ---

def get_theta_translation_2d(theta_0, K, c, num_iter, params_elec, params_gas):
    """Finds theta using the adapted H1 logic (heuristic scalar update)."""
    theta = np.array(theta_0, dtype=float) # Ensure float array, shape (2,)
    
    # Parameters for SGD
    initial_step_size = 0.01 # Needs tuning
    decay_rate = 1000 

    for i in tqdm(range(num_iter), desc="Optimizing theta (Translation)"):
        # Sample from the original distribution
        sample = sample_nig_2d(1, params_elec, params_gas)[0] # Get one (2,) sample
        
        step = initial_step_size / (decay_rate + i)
        
        # Calculate the scalar H1 measure
        h1_val = H1_2d(sample, K, c, theta, params_elec, params_gas)
        
        # Update theta: Since h1_val is scalar, how to update 2D theta?
        # Option 1: Update both components equally (like a magnitude adjustment)
        # Option 2: Need a vector H1.
        # Let's try a simple approach: update based on the sign of theta components?
        # Or maybe use the scalar value to scale a default direction like -theta?
        # Or, use the scalar value as a gradient magnitude applied to each component.
        # This is highly heuristic because H1 was not derived as a gradient magnitude.
        # Let's update both components by the scalar value * direction of -theta
        # This is just one possible heuristic interpretation.
        
        # Avoid division by zero if theta is zero
        norm_theta = np.linalg.norm(theta)
        if norm_theta > 1e-8:
             update_direction = -theta / norm_theta
        else:
             # If theta is near zero, maybe use a random direction or skip update?
             # Or use a fixed direction, e.g., [-1, -1] normalized?
             update_direction = -np.ones_like(theta) / np.sqrt(len(theta)) 

        # Update theta - Note the minus sign was already in original code's update
        # theta_0 -= step * H1(...) 
        # So we use h1_val * update_direction
        update = step * h1_val * update_direction
        
        # Check for NaN/Inf in update
        if np.any(np.isnan(update)) or np.any(np.isinf(update)):
             warnings.warn(f"NaN/Inf in theta update at iteration {i}. Skipping update.")
        else:
             theta -= update # Apply the update

        # Optional: Add projection step here if theta needs to stay in a certain region,
        # although the translation method doesn't have the strict domain like Esscher.
        
    return theta


def get_theta_escher_2d(theta_0, K, c, num_iter, params_elec, params_gas):
    """Finds theta using the adapted H2 logic (Esscher, vector update)."""
    theta = np.array(theta_0, dtype=float) # Ensure float array, shape (2,)
    
    # Parameters for SGD
    initial_step_size = 0.001 # Esscher might need smaller steps, needs tuning
    decay_rate = 1000

    for i in tqdm(range(num_iter), desc="Optimizing theta (Esscher)"):
        
        # Check if current theta is valid before sampling
        beta_e, alpha_e = params_elec['beta'], params_elec['alpha']
        beta_g, alpha_g = params_gas['beta'], params_gas['alpha']
        
        current_beta_e = beta_e + theta[0]
        current_beta_g = beta_g + theta[1]

        # Check domain for sampling: |beta_shifted| < alpha
        # Use strict inequality for safety margin when sampling
        if abs(current_beta_e) >= alpha_e or abs(current_beta_g) >= alpha_g:
             warnings.warn(f"Theta {theta} moved outside CGF domain during optimization. Attempting to project back.")
             # Project theta back to the boundary (or slightly inside)
             # This is a simple projection, might not be optimal
             if abs(current_beta_e) >= alpha_e:
                 theta[0] = np.sign(current_beta_e) * (alpha_e - 1e-6) - beta_e
             if abs(current_beta_g) >= alpha_g:
                 theta[1] = np.sign(current_beta_g) * (alpha_g - 1e-6) - beta_g
             warnings.warn(f"Theta projected to: {theta}")
             # Recompute shifted betas after projection
             current_beta_e = beta_e + theta[0]
             current_beta_g = beta_g + theta[1]
             # Ensure projection worked (might fail if alpha is very small)
             if abs(current_beta_e) >= alpha_e or abs(current_beta_g) >= alpha_g:
                  warnings.warn(f"Projection failed for theta {theta}. Stopping optimization early.")
                  break # Stop optimization

        # Define parameters for the tilted distribution
        params_elec_tilted = params_elec.copy()
        params_elec_tilted['beta'] = current_beta_e
        params_gas_tilted = params_gas.copy()
        params_gas_tilted['beta'] = current_beta_g

        # Sample from the Esscher-tilted distribution
        try:
            sample = sample_nig_2d(1, params_elec_tilted, params_gas_tilted)[0] # Get one (2,) sample
        except (ValueError, RuntimeWarning) as e:
             warnings.warn(f"Error sampling from tilted distribution at iter {i} with theta={theta}. Skipping update. Error: {e}")
             continue # Skip this iteration's update

        # Clip sample to avoid issues in H2
        sample_clipped = np.clip(sample, -700.0, 700.0)

        step = initial_step_size / (decay_rate + i)
        
        # Calculate the H2 vector gradient estimate
        h2_vec = H2_2d(sample_clipped, K, c, theta, params_elec, params_gas) # Pass original params to H2

        # Check for NaN/Inf in gradient estimate
        if np.any(np.isnan(h2_vec)) or np.any(np.isinf(h2_vec)):
             warnings.warn(f"NaN/Inf in H2 gradient estimate at iteration {i}. Skipping update.")
        else:
             # Update theta using the vector gradient estimate
             theta -= step * h2_vec # Apply the update
             
        # Keep theta within the valid domain |beta + theta| < alpha
        # Check after update, project if necessary
        if abs(params_elec['beta'] + theta[0]) >= params_elec['alpha']:
            theta[0] = np.sign(params_elec['beta'] + theta[0]) * (params_elec['alpha'] - 1e-6) - params_elec['beta']
        if abs(params_gas['beta'] + theta[1]) >= params_gas['alpha']:
            theta[1] = np.sign(params_gas['beta'] + theta[1]) * (params_gas['alpha'] - 1e-6) - params_gas['beta']


    # Final check if theta is valid
    if ( abs(params_elec['beta'] + theta[0]) >= params_elec['alpha'] or
         abs(params_gas['beta'] + theta[1]) >= params_gas['alpha'] ):
         warnings.warn(f"Final Esscher theta {theta} is outside or on the boundary of the CGF domain.")
         # Consider returning NaN or the last valid theta? For now, return the potentially invalid one.

    return theta


# --- Monte Carlo Simulation Functions ---

def simulate_mc_crude_2d(num_iter, K, c, params_elec, params_gas):
    """Crude Monte Carlo simulation for the 2D spread option."""
    samples = sample_nig_2d(num_iter, params_elec, params_gas) # Shape (num_iter, 2)
    payoffs = payoff_fn_2d(samples, K, c)
    
    # Check for NaN/Inf in payoffs (less likely after clipping in payoff_fn)
    payoffs = np.nan_to_num(payoffs, nan=0.0, posinf=0.0, neginf=0.0) # Replace with 0
    
    return np.mean(payoffs)


def MC_translation_2d(M, K, c, theta_opt, params_elec, params_gas):
    """Monte Carlo using the translation method (shift sampling distribution)."""
    # Standard Translation IS: Sample Y from pdf(y - theta_opt)
    # Estimate E[payoff(Y) * pdf(Y) / pdf(Y - theta_opt)]
    # Sampling from pdf(y-theta) for NIG means shifting the location parameter mu.
    
    theta_opt = np.asarray(theta_opt) # Ensure numpy array
    
    params_elec_shifted = params_elec.copy()
    params_elec_shifted['mu'] += theta_opt[0]
    params_gas_shifted = params_gas.copy()
    params_gas_shifted['mu'] += theta_opt[1]
    
    total_weighted_payoff = 0.0
    
    for _ in tqdm(range(M), desc="MC Simulation (Translation)"):
        # Sample y from the shifted distribution
        y_sample = sample_nig_2d(1, params_elec_shifted, params_gas_shifted)[0] # Shape (2,)
        
        # Calculate payoff at y
        payoff_y = payoff_fn_2d(y_sample, K, c)
        
        if payoff_y > 0: # Only calculate weight if payoff is non-zero
            # Calculate likelihood ratio: pdf_original(y) / pdf_shifted(y)
            pdf_orig_y = pdf_nig_joint(y_sample, params_elec, params_gas)
            pdf_shifted_y = pdf_nig_joint(y_sample, params_elec_shifted, params_gas_shifted)
            
            # Avoid division by zero or very small numbers
            if pdf_shifted_y < 1e-100:
                weight = 0.0 # Or handle as error / large value? Assume contribution is negligible
                warnings.warn(f"Shifted PDF near zero for sample {y_sample}. Weight set to 0.")
            else:
                weight = pdf_orig_y / pdf_shifted_y
                # Clip weight to prevent extreme values destabilizing the estimate
                weight = np.clip(weight, 0, 1e6) # Upper bound needs tuning
            
            total_weighted_payoff += payoff_y * weight
            
    mean_estimate = total_weighted_payoff / M
    mean_estimate = np.nan_to_num(mean_estimate, nan=0.0, posinf=0.0, neginf=0.0) # Final check
    
    return mean_estimate


def MC_esscher_2d(M, K, c, theta, params_elec, params_gas):
    """Monte Carlo using the Esscher transform."""
    theta = np.asarray(theta) # Ensure numpy array
    beta_e, alpha_e = params_elec['beta'], params_elec['alpha']
    beta_g, alpha_g = params_gas['beta'], params_gas['alpha']

    # Check theta validity for CGF and sampling
    if abs(beta_e + theta[0]) >= alpha_e or abs(beta_g + theta[1]) >= alpha_g:
        raise ValueError(f"Esscher theta {theta} is outside the valid domain |beta+theta|<alpha")

    # Precompute joint CGF value phi(theta)
    psi_val = phi_joint(theta, params_elec, params_gas)
    if np.isnan(psi_val):
        raise ValueError(f"Could not compute CGF for theta {theta}")
        
    # Clip psi_val before exponentiating
    psi_val_clipped = np.clip(psi_val, -700.0, 700.0)
    exp_psi_val = np.exp(psi_val_clipped)

    # Define parameters for the tilted distribution
    params_elec_tilted = params_elec.copy()
    params_elec_tilted['beta'] += theta[0]
    params_gas_tilted = params_gas.copy()
    params_gas_tilted['beta'] += theta[1]

    total_weighted_payoff = 0.0
    for _ in tqdm(range(M), desc="MC Simulation (Esscher)"):
        # Sample x from the Esscher-tilted distribution
        try:
             x = sample_nig_2d(1, params_elec_tilted, params_gas_tilted)[0] # shape (2,)
        except (ValueError, RuntimeWarning) as e:
             warnings.warn(f"Error sampling from tilted distribution during MC with theta={theta}. Skipping sample. Error: {e}")
             continue # Skip this sample

        # Clip sample to prevent overflow in payoff/weight calculation
        x_clipped = np.clip(x, -700.0, 700.0)

        # Calculate payoff
        payoff_x = payoff_fn_2d(x_clipped, K, c)

        # Calculate weight: exp(-theta . x)
        # dot product theta . x = theta[0]*x[0] + theta[1]*x[1]
        dot_prod = np.dot(theta, x_clipped)
        exp_weight_term = np.exp(np.clip(-dot_prod, -700.0, 700.0)) # Clip exponent

        total_weighted_payoff += payoff_x * exp_weight_term
        
    # Average and multiply by exp(psi(theta))
    mean_estimate = (total_weighted_payoff / M) * exp_psi_val
    mean_estimate = np.nan_to_num(mean_estimate, nan=0.0, posinf=0.0, neginf=0.0) # Final check

    return mean_estimate

# --- Example Usage (parameters need to be defined) ---
# Define parameters
params_elec = {'alpha': 2.0, 'beta': 0.2, 'gamma': 0.8, 'mu': 0.04}
params_gas = {'alpha': 1.4, 'beta': 0.2, 'gamma': 0.2, 'mu': 0.04}
K_spread = 1.0 # Example strike
c_gas_coeff = 1.0 # Example coefficient for gas price
num_mc_samples = 300000
num_theta_iter = 300000 # Fewer iterations for optimization might suffice

# # Crude MC
price_crude = simulate_mc_crude_2d(num_mc_samples, K_spread, c_gas_coeff, params_elec, params_gas)
print(f"Crude MC Price: {price_crude}")

# # Esscher Transform
theta0_escher = np.array([0.1, 0.1]) # Initial guess for theta (2D)
theta_opt_escher = get_theta_escher_2d(theta0_escher, K_spread, c_gas_coeff, num_theta_iter, params_elec, params_gas)
print(f"Optimal Esscher Theta: {theta_opt_escher}")
price_escher = MC_esscher_2d(num_mc_samples, K_spread, c_gas_coeff, theta_opt_escher, params_elec, params_gas)
print(f"Esscher MC Price: {price_escher}")

# # Translation Method
# # Note: The H1 adaptation and optimization is heuristic. Results may vary.
theta0_trans = np.array([0.1, 0.1]) # Initial guess
theta_opt_trans = get_theta_translation_2d(theta0_trans, K_spread, c_gas_coeff, num_theta_iter, params_elec, params_gas)
print(f"Optimal Translation Theta: {theta_opt_trans}")
price_trans = MC_translation_2d(num_mc_samples, K_spread, c_gas_coeff, theta_opt_trans, params_elec, params_gas)
print(f"Translation MC Price: {price_trans}")

Crude MC Price: 11.359343682886013


Optimizing theta (Esscher):  21%|████████████████████████▍                                                                                           | 63300/300000 [00:08<00:31, 7441.44it/s]